# Optimal portfolios with frictions

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.optimize import minimize
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions

### Inputs

In [2]:
##### Inputs
# Risk-free rate
RF = 0.02
RF_BORROW = 0.0325

# Expected returns
MNS = np.array([0.06, 0.035])

# Standard deviations
SDS = np.array([0.15, 0.035])

# Correlations
C  = np.identity(2)
C[0, 1] = C[1, 0] = -0.05

# Covariance matrix
COV = np.diag(SDS) @ C @ np.diag(SDS)
print(COV)

[[ 0.0225    -0.0002625]
 [-0.0002625  0.001225 ]]


### Tangency function

In [3]:
def tangency(means, cov, rf):
    n = len(means)
    def f(w):
        mn = w @ means
        sd = np.sqrt(w @ cov @ w)
        return -(mn - rf) / sd
    # Initial guess (equal-weighted)
    w0 = (1/n)*np.ones(n)
    # Constraint: fully-invested portfolio
    A = np.ones(n)
    b = 1
    cons = [{"type": "eq", "fun": lambda x: A @ x - b}]
    # No short-sale constraint
    bnds = [(None, None) for i in range(n)] 
    # Optimization
    TOL = 10**(-10)
    wgts = minimize(f, w0, bounds=bnds, constraints=cons, options={'ftol':TOL}).x
    return wgts

### Tangency portfolio relative to savings rate

In [4]:
wgts_tang_save = tangency(MNS, COV, RF)
print(f'Tangency (relative to savings rate) portfolio weights: {wgts_tang_save}')

Tangency (relative to savings rate) portfolio weights: [0.13203413 0.86796587]


In [5]:
# Portfolio expected return
port_expret = wgts_tang_save @ MNS
print(f'Portfolio Expected Return:\t\t\t {port_expret: ,.4f}')

# Portfolio standard deviation
port_sd = np.sqrt(wgts_tang_save @ COV @ wgts_tang_save)
print(f'Portfolio Standard Deviation:\t\t\t {port_sd: ,.4f}')

# Portfolio sharpe ratio
port_sr = (port_expret - RF)/port_sd
print(f'Portfolio Sharpe Ratio (using savings rate):\t {port_sr: ,.4f}')

Portfolio Expected Return:			  0.0383
Portfolio Standard Deviation:			  0.0354
Portfolio Sharpe Ratio (using savings rate):	  0.5166


### Tangency portfolio relative to borrowing rate

In [6]:
wgts_tang_borrow = tangency(MNS, COV, RF_BORROW)
print(f'Tangency (relative to borrowing rate) portfolio weights: {wgts_tang_borrow}')

Tangency (relative to borrowing rate) portfolio weights: [0.35111804 0.64888196]


In [7]:
# Portfolio expected return
port_expret = wgts_tang_borrow @ MNS
print(f'Portfolio Expected Return:\t\t\t {port_expret: ,.4f}')

# Portfolio standard deviation
port_sd = np.sqrt(wgts_tang_borrow @ COV @ wgts_tang_borrow)
print(f'Portfolio Standard Deviation:\t\t\t {port_sd: ,.4f}')

# Portfolio sharpe ratio
port_sr = (port_expret - RF_BORROW)/port_sd
print(f'Portfolio Sharpe Ratio (using savings rate):\t {port_sr: ,.4f}')

Portfolio Expected Return:			  0.0438
Portfolio Standard Deviation:			  0.0563
Portfolio Sharpe Ratio (using savings rate):	  0.2003


### Capital allocation and risk aversion

- Method #1: Find risk-aversions that represent each tangency portfolio
- Method #2: Directly maximize MV utility using all assets

#### Method 1
Set $w^*=1$ in the expression $w^* = \frac{E[r_p-r_f]}{A\cdot \text{var}(r_p)}$ and solve for $A$:

$$A = \frac{E[r_p-r_f]}{\text{var}(r_p)}.$$


In [8]:
lo_expret = wgts_tang_save @ MNS
lo_var    = wgts_tang_save @ COV @ wgts_tang_save
raver_upper_threshold = (lo_expret - RF) / lo_var
print(f'Risk aversion upper threshold:\t {raver_upper_threshold: ,.1f}')
print(f'Risk aversions above this level will do some saving')

Risk aversion upper threshold:	  14.6
Risk aversions above this level will do some saving


In [9]:
hi_expret = wgts_tang_borrow @ MNS
hi_var    = wgts_tang_borrow @ COV @ wgts_tang_borrow
raver_lower_threshold = (hi_expret - RF_BORROW) / hi_var
print(f'Risk aversion lower threshold:\t {raver_lower_threshold: ,.1f}')
print(f'Risk aversions below this level will invest in levered positions')
print(f'Risk aversions between {raver_lower_threshold: ,.1f} and {raver_upper_threshold: ,.1f} will invest along the risky asset frontier')


Risk aversion lower threshold:	  3.6
Risk aversions below this level will invest in levered positions
Risk aversions between  3.6 and  14.6 will invest along the risky asset frontier


For risk aversions between the thresholds, we solve for the frontier portfolio that maximizes mean-variance utility at the given risk aversion.  We make use of the fact that any frontier portfolio can be expressed as a portfolio of two frontier portfolios.  So we consider the portfolio that invests $a$ in the efficient low-risk portfolio and $1-a$ in the efficient high-risk portfolio and maximizes utility:
$$ \underset{a}\max E[r_p] - 0.5\cdot A \cdot \text{var}[r_p]\,. $$  
with $r_p = a \cdot r_{\text{LO}} + (1-a) \cdot r_{\text{HI}}$.  In order to calculate the portfolio variance, we need to calculate the covariance between the two frontier portfolios.  


In [10]:
# What is the covariance between the efficient low and high risk portfolios?
cov_hilo = wgts_tang_save @ COV @ wgts_tang_borrow

In [11]:
def opt_allocation(means, cov, rf_save, rf_borrow, risk_aversion):
    # Calculate two tangencies
    wgts_tang_save = tangency(means, cov, rf_save)
    wgts_tang_borrow = tangency(means, cov, rf_borrow)

    # Find risk aversion thresholds
    lo_expret = wgts_tang_save @ means
    lo_var    = wgts_tang_save @ cov @ wgts_tang_save
    raver_upper_threshold = (lo_expret - rf_save) / lo_var  

    hi_expret = wgts_tang_borrow @ means
    hi_var    = wgts_tang_borrow @ cov @ wgts_tang_borrow
    raver_lower_threshold = (hi_expret - rf_borrow) / hi_var    
    
    # Find optimal allocation based on risk aversion
    wgts = np.zeros(4)  # 1st element is risk-free saving, 2nd element is risk-free borrowing, 3nd is low-risk tangency, 4th
    if risk_aversion >= raver_upper_threshold:
        # Savings CAL
        wgts[2] = (lo_expret - rf_save) / (risk_aversion * lo_var)
        wgts[0] = 1-wgts[2]
        wgts[1] = 0.0
        wgts[3] = 0.0
    elif risk_aversion >= raver_lower_threshold:
        # Risky asset frontier
        cov_hilo = wgts_tang_save @ cov @ wgts_tang_borrow
        wgts[0] = 0.0
        wgts[1] = 0.0
        wgts[2] = (lo_expret-hi_expret - risk_aversion * (cov_hilo  - hi_var))/(risk_aversion*(lo_var + hi_var - 2*cov_hilo))
        wgts[3] = 1-wgts[2]
    else:
        # Borrowing CAL
        wgts[0] = 0.0  
        wgts[2] = 0.0  
        wgts[3] = (hi_expret - rf_borrow) / (risk_aversion * hi_var)
        wgts[1] = 1-wgts[3]
    return wgts


In [12]:
RAVER = 2
wgts = opt_allocation(MNS, COV, RF, RF_BORROW, RAVER)
print(f'For risk aversion of {RAVER: .1f}, invest:')
print(f'\tRisk-free saving\t{wgts[0]: ,.1%}')
print(f'\tRisk-free borrowing\t{wgts[1]: ,.1%}')
print(f'\tEfficient low-risk\t{wgts[2]: ,.1%}')
print(f'\tEfficient high-risk\t{wgts[3]: ,.1%}')


For risk aversion of  2.0, invest:
	Risk-free saving	 0.0%
	Risk-free borrowing	-77.9%
	Efficient low-risk	 0.0%
	Efficient high-risk	 177.9%


In [13]:
# Expected returns for risk-free savings + borrowing + low-risk efficient + high-risk efficient
new_mns = np.array([RF, RF_BORROW, lo_expret, hi_expret])
new_mns

array([0.02      , 0.0325    , 0.03830085, 0.04377795])

In [14]:
# COV for risk-free savings + borrowing + low-risk efficient + high-risk efficient
new_cov = np.zeros((4,4))
new_cov[2,2] = lo_var
new_cov[3,3] = hi_var
new_cov[2,3] = new_cov[3,2] = cov_hilo
new_cov

array([[0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.00125495, 0.00163053],
       [0.        , 0.        , 0.00163053, 0.00317006]])

In [15]:
# Portfolio expected return
optport_expret = wgts @ new_mns
print(f'For risk aversion of {RAVER: .1f}, Portfolio Expected Return:\t\t {optport_expret: ,.4f}')

# Portfolio standard deviation
optport_sd     = np.sqrt(wgts @ new_cov @ wgts)
print(f'For risk aversion of {RAVER: .1f}, Portfolio Standard Deviation:\t {optport_sd: ,.4f}')

For risk aversion of  2.0, Portfolio Expected Return:		  0.0526
For risk aversion of  2.0, Portfolio Standard Deviation:	  0.1002


#### Method #2: Directly maximize MV utility using all assets

The optimal portfolio for investor with risk aversion $A$:
$$ \underset{w_{\text{saving}},w_{\text{borrow}},w_1,w_2,\dots,w_N}{\text{max}} E[r_p] - 0.5 \cdot A \cdot \text{var}[r_p] $$ 
subject to the constraints $w_{\text{saving}} + w_{\text{borrow}} + \sum_i w_i=1$, $w_{\text{saving}} \ge 0$, and $w_{\text{borrow}} \le 0$. 

We will map this problem into the `cvxopt.solvers.qp` function's general form:
\begin{align*}
    \underset{w}{\text{min  }}& \frac{1}{2} w' Q w + p'w \\
     \text{subject to  } & Gw \le h \\
                        & Aw = b \\
\end{align*}

In [16]:
def opt_allocation2(means, cov, rf_save, rf_borrow, risk_aversion):
    n=len(means)
    Q = np.zeros((n + 2, n + 2))
    Q[2:, 2:] = risk_aversion * cov
    Q = matrix(Q, tc="d")
    p = np.array([-rf_save, -rf_borrow] + list(-means))
    p = matrix(p, (n + 2, 1), tc="d")
    # Constraint: saving weight positive, borrowing weight negative
    G = np.zeros((2, n + 2))
    G[0, 0] = -1
    G[1, 1] = 1
    G = matrix(G, (2, n+2), tc="d")
    h = matrix([0, 0], (2, 1), tc="d")
    # Constraint: fully-invested portfolio
    A = matrix(np.ones(n+2), (1, n+2), tc="d")
    b = matrix([1], (1, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b)
    if sol["status"] == "optimal":
        wgts_optimal = np.array(sol["x"]).flatten()
    else:
        wgts_optimal = None
    return wgts_optimal

In [17]:
wgts = opt_allocation2(MNS, COV, RF, RF_BORROW, RAVER)
print(f'For risk aversion of {RAVER: .1f}, invest:')
print(f'\tRisk-free saving\t\t{wgts[0]: ,.1%}')
print(f'\tRisk-free borrowing\t\t{wgts[1]: ,.1%}')
print(f'\tTotal weight in risky assets:\t{np.sum(wgts[2:]): .1%}')
print(f'Weights in each risky asset of {wgts[2:].round(4)}')


     pcost       dcost       gap    pres   dres
 0: -4.2049e-02 -2.9322e-02  2e+00  1e+00  1e+00
 1: -2.9913e-02 -6.4796e-02  4e-02  2e-02  2e-02
 2: -3.9174e-02 -4.2633e-02  3e-03  6e-05  6e-05
 3: -4.2497e-02 -4.2538e-02  4e-05  7e-07  7e-07
 4: -4.2530e-02 -4.2531e-02  4e-07  7e-09  7e-09
 5: -4.2531e-02 -4.2531e-02  4e-09  7e-11  7e-11
Optimal solution found.
For risk aversion of  2.0, invest:
	Risk-free saving		 0.0%
	Risk-free borrowing		-77.9%
	Total weight in risky assets:	 177.9%
Weights in each risky asset of [0.6246 1.1542]


In [18]:
# Augment undelying asset means and covariances
augmented_mns = np.append([RF, RF_BORROW], MNS)
n = len(MNS)
augmented_cov = np.zeros((n + 2, n + 2))
augmented_cov[2:,2:] = COV

# Portfolio expected return
optport_expret = wgts @ augmented_mns
print(f'For risk aversion of {RAVER: .1f}, Portfolio Expected Return:\t\t {optport_expret: ,.4f}')

# Portfolio standard deviation
optport_sd     = np.sqrt(wgts @ augmented_cov @ wgts)
print(f'For risk aversion of {RAVER: .1f}, Portfolio Standard Deviation:\t {optport_sd: ,.4f}')

For risk aversion of  2.0, Portfolio Expected Return:		  0.0526
For risk aversion of  2.0, Portfolio Standard Deviation:	  0.1002


### Optimal allocation dataframe

In [19]:
RAVER_LEVELS = np.arange(0.25,30.25,0.25)

# Dataframe
df_opt = pd.DataFrame(dtype='float',columns=['risk_aversion','saving','borrowing','low_risk','high_risk','port_expret', 'port_sd'], index=np.arange(len(RAVER_LEVELS)))
df_opt.risk_aversion = RAVER_LEVELS
for i in df_opt.index:
    wgts = opt_allocation(MNS, COV, RF, RF_BORROW, df_opt.loc[i,'risk_aversion'])
    df_opt.loc[i,['saving','borrowing','low_risk','high_risk']]=wgts
    df_opt.loc[i,'port_expret']= wgts @ new_mns
    df_opt.loc[i,'port_sd']    = np.sqrt(wgts @ new_cov @ wgts)
df_opt.head()

,risk_aversion,saving,borrowing,low_risk,high_risk,port_expret,port_sd
0,0.25,0.0,-13.230593,0.0,14.230593,0.192992,0.801229
1,0.50,0.0,-6.115297,0.0,7.115297,0.112746,0.400614
2,0.75,0.0,-3.743531,0.0,4.743531,0.085997,0.267076
3,1.00,0.0,-2.557648,0.0,3.557648,0.072623,0.200307
4,1.25,0.0,-1.846119,0.0,2.846119,0.064598,0.160246


### The full picture
Note we will now also calculate the sharpe ratio for each frontier portfolio

In [20]:
# Calculate frontier portfolios (from last time)
def frontier(means, cov, target):
    n = len(means)
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    # Constraint: short-sales allowed
    G = matrix(np.zeros((n,n)), tc="d")
    h = matrix(np.zeros(n), (n, 1), tc="d")
    # Fully-invested constraint + E[r] = target
    A = matrix(np.vstack((np.ones(n), means)), (2, n), tc="d")
    b = matrix([1, target], (2, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b)
    wgts = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts
SolverOptions['show_progress'] = False

NUM_TARGETS = 30
df = pd.DataFrame(dtype='float',columns=['target_expret','w1','w2','port_expret','port_sd','sharpe'], index=np.arange(NUM_TARGETS))
df.target_expret = np.linspace(MNS.min(), MNS.max(),NUM_TARGETS)
for i in df.index:
    wgts = frontier(MNS, COV, df.loc[i,'target_expret'])
    df.loc[i,['w1','w2']] = wgts
    df.loc[i,'port_expret'] = wgts @ MNS
    df.loc[i,'port_sd'] = np.sqrt(wgts @ COV @ wgts)
df.sharpe = (df.port_expret - RF)/df.port_sd

In [24]:
fig = go.Figure()

# Plot frontier
string =  "Stock: %{customdata[0]:.1%}<br>"
string += "Bond: %{customdata[1]:.1%}<br>"
string += "<extra></extra>"
trace= go.Scatter(x=df.port_sd, y=df.port_expret,mode="lines",
    customdata=df[['w1','w2']],hovertemplate=string, name='Frontier')
fig.add_trace(trace)

# Plot underlying assets
trace1= go.Scatter(x=[SDS[0]], y=[MNS[0]],mode="markers", marker=dict(size=10, color="red"), name='Stock')
fig.add_trace(trace1)
trace2= go.Scatter(x=[SDS[1]], y=[MNS[1]],mode="markers", marker=dict(size=10, color="red"), name='Bond')
fig.add_trace(trace2)

# Plot low-risk tangency
trace= go.Scatter(x=[np.sqrt(lo_var)], y=[lo_expret] ,
    mode="markers", marker=dict(size=10, color="black"),
    customdata=wgts_tang_save.reshape(1,2),hovertemplate=string, name='Efficient Low-Risk Tangency')
fig.add_trace(trace)

# Plot low-risk tangency
trace= go.Scatter(x=[np.sqrt(hi_var)], y=[hi_expret] ,
    mode="markers", marker=dict(size=10, color="black"),
    customdata=wgts_tang_borrow.reshape(1,2),hovertemplate=string, name='Efficient High-Risk Tangency')
fig.add_trace(trace)

# Plot optimal allocation
string =  "Saving: %{customdata[0]:.1%}<br>"
string += "Borrowing: %{customdata[1]:.1%}<br>"
string += "Efficient Low-Risk: %{customdata[2]:.1%}<br>"
string += "Efficient High-Risk: %{customdata[3]:.1%}<br>"
string += "Optimal if risk aversion is: %{customdata[4]:.1f}<br>"
string += "<extra></extra>"
trace= go.Scatter(x=df_opt.port_sd, y=df_opt.port_expret,mode="lines",marker=dict(color="black"),
    customdata=df_opt[['saving','borrowing','low_risk','high_risk','risk_aversion']],hovertemplate=string, name='Optimal allocation')
fig.add_trace(trace)

# Formatting
fig.layout.yaxis["title"] = "Expected Return"
fig.layout.xaxis["title"] = "Standard Deviation"
fig.update_yaxes(tickformat=".1%")
fig.update_xaxes(tickformat=".1%")
fig.update_xaxes(range=[0.5 * df["port_sd"].min(), 1.25 * df["port_sd"].max()])
fig.update_yaxes(range=[0.7 * df["port_expret"].min(), 1.25 * df["port_expret"].max()])
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()